In [ ]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))


import csv
from datetime import datetime
from datetime import timedelta
import os
import gc
import scipy
import numpy as np
from matplotlib import pyplot as plt
import sys
import pandas as pd
import random
import time
from sklearn import preprocessing
from tqdm import tqdm
from scipy import signal
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import seaborn as sns
import math
import itertools
from scipy.optimize import minimize

from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact
from functools import partial

from pre_utils import autocorrelation, lowpass, moving_average

In [ ]:

def scaling_quarter_range(data):
    data_quarter = np.zeros(data.shape)
    for i in range(data.shape[0]):
        q25,q75 = np.percentile(data[i],[25,75])
        for j in range(data.shape[1]):
            x = data[i][j]
            new_x = (x-q25)/(q75-q25)
            data_quarter[i][j] = new_x
    return data_quarter

In [ ]:

def calc_autoco_maxid(acc,sampling_rate,cut_off):
    len_acc = acc.shape[1]
    # 平滑化（ローパスフィルタ）
    lpf = np.zeros((3,len_acc))
    for i in range(3):
        lpf[i] = lowpass(sampling_rate,acc[i],cut_off)

#     lpf_mm = scaling_quarter_range(lpf)
#     lpf_mm = preprocessing.minmax_scale(lpf.astype(float),axis=1)

    one_dim = np.sqrt(lpf[0]**2+lpf[1]**2+lpf[2]**2)

    auto_co = []
    for k in range(len_acc):
        auto_co.append(autocorrelation(one_dim,k))
    auto_co = np.array(auto_co)

    maxid = signal.argrelmax(auto_co,order=50)
    peak = maxid[0]
    peak_val = [auto_co[p] for p in peak]
    
    if len(peak_val)>0:
        peak_max_val = np.max(peak_val)
        peak_max_idx = peak[np.argmax(peak_val)]
    elif len(peak_val)==0:
        peak_max_val = 0
        peak_max_idx = 0
    
    return peak_max_val,peak_max_idx,auto_co

In [ ]:

def calc_peak_autoco(acc,sampling_rate,cut_off):
    len_acc = acc.shape[1]
    acc_arr = []
    peak_arr = []
    idx_arr = []
    ac_arr = []

    #full
    full_acc = acc
    acc_arr.append(full_acc)
    peak,idx,ac = calc_autoco_maxid(full_acc,sampling_rate,cut_off)
    peak_arr.append(peak)
    idx_arr.append(idx)
    ac_arr.append(ac)

    #half
    half_len = int(len_acc/2)
    for i in range(0,len_acc-half_len,100):
        half_acc = acc[:,i:i+half_len]
        acc_arr.append(half_acc)
        peak,idx,ac = calc_autoco_maxid(half_acc,sampling_rate,cut_off)
        peak_arr.append(peak)
        idx_arr.append(idx)
        ac_arr.append(ac)

    #third
    third_len = int(len_acc/3)
    for i in range(0,len_acc-third_len,100):
        third_acc = acc[:,i:i+third_len]
        acc_arr.append(third_acc)
        peak,idx,ac = calc_autoco_maxid(third_acc,sampling_rate,cut_off)
        peak_arr.append(peak)
        idx_arr.append(idx)
        ac_arr.append(ac)

    # quater
    quater_len = int(len_acc/4)
    for i in range(0,len_acc-quater_len-1,int(quater_len/2)):
        quater_acc = acc[:,i:i+quater_len]
        acc_arr.append(quater_acc)
        peak,idx,ac = calc_autoco_maxid(quater_acc,sampling_rate,cut_off)
        peak_arr.append(peak)
        idx_arr.append(idx)
        ac_arr.append(ac)
        
    return acc_arr,peak_arr,idx_arr,ac_arr

In [ ]:

def calc_DTW_sequence(acc,cycle_len):
    dtw_x_sum = []
    dtw_y_sum = []
    dtw_z_sum = []
    
    len_acc = acc.shape[1]

    for x in range(len_acc-num*cycle_len):
        segments = []
        for i in range(num):
            seg = acc[:,x+cycle_len*i:x+cycle_len*(i+1)]
            segments.append(seg)

        pairs = list(itertools.combinations(segments, 2))
        dtw_x = 0
        dtw_y = 0
        dtw_z = 0
        for p in pairs:
            distance_x,path = fastdtw(p[0][0],p[1][0])
            distance_y,path = fastdtw(p[0][1],p[1][1])
            distance_z,path = fastdtw(p[0][2],p[1][2])
            dtw_x += distance_x
            dtw_y += distance_y
            dtw_z += distance_z

        dtw_x_sum.append(dtw_x)
        dtw_y_sum.append(dtw_y)
        dtw_z_sum.append(dtw_z)

    dtw_sum = np.array(dtw_x_sum)+np.array(dtw_y_sum)+np.array(dtw_z_sum)
    return dtw_sum

In [ ]:
def decide_start_point(acc,idx,minima,cycle,num):
    if len(minima)!= 0:
        min_idx = np.argmin(minima)
        sp = idx[min_idx]
        std_segs = []
        for i in range(num):
            seg = acc[:,sp+cycle*i:sp+cycle*(i+1)]
            std_seg = 0
            for j in range(3):
                std_seg += np.std(seg[j])
            std_segs.append(std_seg)
        if np.min(std_segs) >= 1.5:
            return sp
        elif np.min(std_segs) < 1.5:
            minima.pop(min_idx)
            idx.pop(min_idx)
            new_sp = decide_start_point(acc,idx,minima,cycle,num)
            return new_sp
    else: return 0

In [ ]:

def calc_segment_DTW(start_and_length,acc,num):
#     print(start_and_length)
    segments = []
    for i in range(num):
        seg = acc[:,int(start_and_length[i]):int(start_and_length[i]+start_and_length[i+num])]
        segments.append(seg)
        
    pairs = list(itertools.combinations(segments, 2))
    
    dtw_x = 0
    dtw_y = 0
    dtw_z = 0
    for p in pairs:
        distance_x,path = fastdtw(p[0][0],p[1][0])
        distance_y,path = fastdtw(p[0][1],p[1][1])
        distance_z,path = fastdtw(p[0][2],p[1][2])
        dtw_x += distance_x
        dtw_y += distance_y
        dtw_z += distance_z
    dtw_sum = dtw_x + dtw_y + dtw_z
    return dtw_sum

In [ ]:

dt = 0.01
sampling_rate = 100

cut_off = 2


subject = 'A'
action = 'pushup'
num = 2

### Loading Data

In [ ]:
time1 = time.time()

acc = np.load('../data/few-shot_data/'+subject+'/'+str(num)+'times/'+action+'.npy')
len_acc = acc.shape[1]

fig = plt.figure(figsize=(15,5))
plt.plot(acc.T)


acc_mm_quarter = scaling_quarter_range(acc)

In [ ]:
opt_func = partial(calc_segment_DTW,acc=acc_mm_quarter,num=num)

In [ ]:
acc_arr,peak_arr,idx_arr,ac_arr = calc_peak_autoco(acc_mm_quarter,sampling_rate,cut_off)
peak_arr = np.array(peak_arr)
idx_arr = np.array(idx_arr)
peak_arg = np.argsort(-peak_arr)

peak_sorted = peak_arr[peak_arg]
idx_sorted = idx_arr[peak_arg]

In [ ]:
cycle_can = list(set(idx_sorted[:5]))
print(cycle_can)

time2 = time.time()

In [ ]:
dtw_arr = []
init_arr = []
for c in cycle_can:
    dtw_sum = calc_DTW_sequence(acc_mm_quarter,c)
    minima = dtw_sum[signal.argrelmin(dtw_sum,order=100)].tolist()
    idx = signal.argrelmin(dtw_sum,order=100)[0].tolist()
    start_point = decide_start_point(acc,idx,minima,c,num)
    if start_point == None:
        continue
    starts0 = [start_point + c * i for i in range(num)]
    lengths0 = [c for i in range(num)]
    
    opt_init = starts0+lengths0
    init_arr.append(opt_init)
    dtw_before = opt_func(opt_init)
    dtw_arr.append(dtw_before)

In [ ]:
plt.plot(dtw_sum)
plt.tick_params(bottom=False,
               left=False,
               right=False,
               top=False,)

In [ ]:
min_dtw = np.argmin(dtw_arr)
cycle_len = cycle_can[min_dtw]
opt_init = init_arr[min_dtw]
dtw_before = opt_func(opt_init)

print('length of action:',cycle_len)
print('initial value of optimize function:',opt_init)
print('initial DTW value :',dtw_before)

In [ ]:
starts_init = np.array(opt_init[0:num])
length_init = np.array(opt_init[num:num*2])
ends_init = starts_init + length_init

fig = plt.figure(figsize=(20,5))
plt.plot(acc.T)
plt.vlines(starts_init,np.min(acc),np.max(acc),color='black',label='start')
# plt.xticks(starts_opt,rotation=90)
plt.vlines(ends_init,np.min(acc),np.max(acc),color='red',label='end')
# plt.xticks(ends_opt,rotation=90)
plt.title('base of start and end point')
plt.legend()

time3 = time.time()

In [ ]:

diff_start = cycle_len*0.5
diff_len = cycle_len*0.2

bound_start = [(max(0,starts_init[i]-diff_start),min(len_acc,starts_init[i]+diff_start)) for i in range(num)]
bound_len = [(cycle_len-diff_len,cycle_len+diff_len) for i in range(num)]

boundary = bound_start+bound_len

### Optimization

In [ ]:
fun_arr = []
x_arr = []
for e in range(10):
    res = minimize(opt_func,opt_init,method='L-BFGS-B',bounds=boundary,options={'eps':e,'iprint':1})
    fun_arr.append(res.fun)
    x_arr.append(res.x)

In [ ]:
min_dtw_idx = np.argmin(fun_arr)
opted_para = x_arr[min_dtw_idx]

In [ ]:
starts_opt = np.array(opted_para[0:num])
length_opt = np.array(opted_para[num:num*2])
ends_opt = starts_opt + length_opt

fig = plt.figure(figsize=(20,5))
plt.plot(acc.T)
plt.vlines(starts_opt,np.min(acc),np.max(acc),color='black',label='start')
# plt.xticks(starts_opt,rotation=90)
plt.vlines(ends_opt,np.min(acc),np.max(acc),color='red',label='end')
# plt.xticks(ends_opt,rotation=90)
plt.title('base of start and end point')
plt.legend()

time4 = time.time()

### Extracting

In [ ]:

base_seg = {}
for i in range(num):
    seg = acc[:,int(starts_opt[i]):int(ends_opt[i])]
    base_seg[i] = seg
fig,axes = plt.subplots(1,num,figsize=(num*5,3))
for i in range(num):
    axes[i].plot(base_seg[i].T)
    axes[i].set_title(str(i+1)+'th action')

In [ ]:
make_num = 100
r = 0.1
base_path = '../data/one_time_action_opt/'
make_path = base_path+subject+'/'+action+'/'+str(num)

if not os.path.isdir(make_path):
    os.makedirs(make_path)

In [ ]:
for i in range(num):
    for j in range(make_num):
        start_r = random.randint(int(-0.1*length_opt[i]),int(0.1*length_opt[i]))
        end_r = random.randint(int(-0.1*length_opt[i]),int(0.1*length_opt[i]))
        sp = starts_opt[i] - start_r
        ep = ends_opt[i] + end_r
        seg = acc[:,int(sp):int(ep)]
        np.save(make_path+'/'+str(i+1)+'_'+str(j),seg)

In [ ]:
begin = acc[:,:int(starts_opt[0])]
fin = acc[:,int(ends_opt[-1]):]

fig,axes = plt.subplots(1,2,figsize=(10,3))
axes[0].plot(begin.T)
axes[0].set_title('pre action')
axes[1].plot(fin.T)
axes[1].set_title('after action')

In [ ]:
np.save(make_path+'/begin.npy',begin)
np.save(make_path+'/fin.npy',fin)

time5 = time.time()